In [2]:

!pip install playwright

In [3]:

import subprocess

process = subprocess.run(['playwright', 'install'], capture_output=True, text=True)
print(process.stdout)
print(process.stderr)

|                                                                                |   0% of 164.7 MiB
|■■■■■■■■                                                                        |  10% of 164.7 MiB
|■■■■■■■■■■■■■■■■                                                                |  20% of 164.7 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■                                                        |  30% of 164.7 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                                |  40% of 164.7 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                        |  50% of 164.7 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                                |  60% of 164.7 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                        |  70% of 164.7 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■                |  80% of 164.7 MiB
|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■        |  90% of 

In [4]:

import subprocess

process_deps = subprocess.run(['apt-get', 'update'], capture_output=True, text=True)
print(process_deps.stdout)
print(process_deps.stderr)

process_deps = subprocess.run(['apt-get', 'install', '-y', 'libxcomposite1', 'libgtk-3-0', 'libatk1.0-0'], capture_output=True, text=True)
print(process_deps.stdout)
print(process_deps.stderr)

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cli.github.com/packages stable/main amd64 Packages [354 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,624 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [38.5 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,871 kB]
Get:14 http

In [5]:

import re
from datetime import datetime
from io import StringIO

import requests
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import asyncio

MLS_URL = "https://www.mlssoccer.com/news/fifa-world-cup-2026-schedule-every-game-by-city-stadium"
LATLONG_URL = "https://www.latlong.net/location/2026-fifa-world-cup-locations-2252"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

MONTH_RE = re.compile(r"^(?:[\u2022•\*\-\u2013\u2014]*\s*)?(June|July)\s+(\d{1,2}):\s*(.*)$")

STADIUM_HDR_RE = re.compile(r"^(?P<city>.+?)\s*[\-\u2013\u2014]\s*(?P<stadium>.+)$")

COUNTRIES = {"United States", "Canada", "Mexico"}

STADIUM_SYNONYMS = {
    "Estadio Azteca": "Estadio Banorte",
}

PACKED_LATLON_RE = re.compile(r"^\s*([+\-]?\d+(?:\.\d+)?)([+\-]\d+(?:\.\d+)?)\s*$")

def http_get(url: str) -> str:
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text

async def http_get_playwright(url: str) -> str:
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, wait_until='domcontentloaded', timeout=60000)

        try:
            await page.locator("button:has-text('Accept', 'Consent')").first.click(timeout=5000)
            print("Clicked cookie consent button (if present).")
        except Exception as e:
            print(f"No cookie consent button found or clickable: {e}")

        await page.wait_for_selector('main', timeout=60000)
        await page.wait_for_timeout(5000)

        html_content = await page.content()
        await browser.close()
        return html_content

def parse_packed_latlon(s: str):
    """
    Parse strings like '32.748138-97.093231' -> (32.748138, -97.093231)
    Returns (None, None) if not parsable.
    """
    s = str(s).strip()
    m = PACKED_LATLON_RE.match(s)
    if not m:
        return None, None
    return float(m.group(1)), float(m.group(2))


def parse_latlong_coords(html: str) -> pd.DataFrame:
    """
    Returns DataFrame: stadium, lat, lon
    """
    tables = pd.read_html(StringIO(html))
    if not tables:
        raise RuntimeError("No tables found on LatLong page.")

    t = tables[0].copy()
    t.columns = [str(c).strip().lower() for c in t.columns]

    name_col = None
    for cand in ["location name", "location", "name"]:
        if cand in t.columns:
            name_col = cand
            break
    if name_col is None:
        name_col = t.columns[0]

    lat_col = "latitude" if "latitude" in t.columns else t.columns[1]
    lon_col = "longitude" if "longitude" in t.columns else None

    rows = []
    for _, row in t.iterrows():
        stadium = str(row[name_col]).strip()

        if lon_col is not None and pd.notna(row.get(lon_col, None)):
            lat_raw = str(row[lat_col]).strip()
            lon_raw = str(row[lon_col]).strip()

            try:
                lat = float(lat_raw)
                lon = float(lon_raw)
            except Exception:
                lat, lon = parse_packed_latlon(lat_raw)

        else:
            lat_raw = str(row[lat_col]).strip()
            lat, lon = parse_packed_latlon(lat_raw)

        if lat is not None and lon is not None:
            rows.append((stadium, lat, lon))

    coords = pd.DataFrame(rows, columns=["stadium", "lat", "lon"])
    coords = coords.drop_duplicates(subset=["stadium"]).reset_index(drop=True)

    if coords.empty:
        raise RuntimeError("Parsed 0 stadium coordinates from LatLong page (page format may have changed).")

    return coords


def parse_mls_matches(html: str) -> pd.DataFrame:
    """
    Returns DataFrame with columns including:
      date, phase, stage_detail, group, match, city, state_region, country, stadium, stadium_key_for_coords
    """
    soup = BeautifulSoup(html, "html")
    main = soup.find("main") or soup
    text = main.get_text("\n")

    lines = [ln.replace("\xa0", " ").strip() for ln in text.splitlines()]
    lines = [ln for ln in lines if ln]

    current_city = None
    current_state = None
    current_country = None
    current_stadium = None
    current_phase = None

    rows = []
    i = 0
    while i < len(lines):
        ln = lines[i]

        m_hdr = STADIUM_HDR_RE.match(ln)
        if m_hdr and any(k in m_hdr.group("stadium") for k in ["Stadium", "Field", "Place", "Estadio"]):
            city_part = m_hdr.group("city").strip()
            stadium_part = m_hdr.group("stadium").strip()

            if "," in city_part:
                current_city, current_state = [x.strip() for x in city_part.split(",", 1)]
            else:
                current_city, current_state = city_part, None

            current_stadium = stadium_part
            current_phase = None
            current_country = None

            j = i + 1
            while j < len(lines) and \
                  not lines[j].strip() in COUNTRIES and \
                  not STADIUM_HDR_RE.match(lines[j]) and \
                  lines[j].strip() not in ["Group Stage", "Knockout Stage"]:
                j += 1

            if j < len(lines) and lines[j].strip() in COUNTRIES:
                current_country = lines[j].strip()
                i = j + 1
            else:
                i += 1

            continue

        if ln == "Group Stage":
            current_phase = "Group Stage"
            i += 1
            continue
        if ln == "Knockout Stage":
            current_phase = "Knockout Stage"
            i += 1
            continue

        m = MONTH_RE.match(ln)
        if m and current_stadium and current_city:
            month = m.group(1)
            day = int(m.group(2))
            rest_of_date_line = m.group(3).strip()

            match_description_to_parse = rest_of_date_line
            increment_i_by = 1

            if not match_description_to_parse and (i + 1 < len(lines)):
                potential_match_desc_next_line = lines[i + 1].strip()
                if not STADIUM_HDR_RE.match(potential_match_desc_next_line) \
                   and potential_match_desc_next_line not in COUNTRIES \
                   and potential_match_desc_next_line not in ["Group Stage", "Knockout Stage"] \
                   and not MONTH_RE.match(potential_match_desc_next_line):
                    match_description_to_parse = potential_match_desc_next_line
                    increment_i_by = 2

            if match_description_to_parse:
                dt = datetime.strptime(f"{month} {day} 2026", "%B %d %Y").date().isoformat()

                group = None
                stage_detail = None
                match_desc = None

                if " | " in match_description_to_parse:
                    match_desc, group = [x.strip() for x in match_description_to_parse.split(" | ", 1)]
                    stage_detail = group
                else:
                    par = re.search(r"\(([^)]+)\)\s*$", match_description_to_parse)
                    if par:
                        stage_detail = par.group(1).strip()
                        match_desc = match_description_to_parse[:par.start()].strip()
                    else:
                        match_desc = match_description_to_parse

                stadium_key = STADIUM_SYNONYMS.get(current_stadium, current_stadium)

                rows.append({
                    "date": dt,
                    "phase": current_phase,
                    "stage_detail": stage_detail,
                    "group": group,
                    "match": match_desc,
                    "city": current_city,
                    "state_region": current_state,
                    "country": current_country,
                    "stadium": current_stadium,
                    "stadium_key_for_coords": stadium_key,
                })

            i += increment_i_by
            continue

        i += 1

    df = pd.DataFrame(rows)

    if df.empty:
        preview = "\n".join(lines[:30])
        raise RuntimeError(
            "Parsed 0 matches from MLS page.\n"
            "This usually means either:\n"
            "  (1) the page HTML returned is a consent/block/interstitial page, or\n"
            "  (2) MLS changed their formatting.\n\n"
            "First lines seen:\n"
            f"{preview}"
        )

    return df


async def main_async(
    out_matches_csv="worldcup2026_matches_with_stadium_coords.csv",
    out_coords_csv="worldcup2026_stadium_coords.csv",
    also_write_databricks_filestore=False
):
    mls_html = await http_get_playwright(MLS_URL)
    print("--- Start MLS HTML Content ---")
    print(mls_html[:1000])
    print("--- End MLS HTML Content ---")

    latlong_html = http_get(LATLONG_URL)

    matches = parse_mls_matches(mls_html)
    coords = parse_latlong_coords(latlong_html)

    coords_out = coords.sort_values("stadium").reset_index(drop=True)
    coords_out.to_csv(out_coords_csv, index=False)

    merged = matches.merge(
        coords_out,
        left_on="stadium_key_for_coords",
        right_on="stadium",
        how="left",
        suffixes=("", "_coords"),
    ).drop(columns=["stadium_coords"], errors="ignore") \
     .rename(columns={"lat": "stadium_lat", "lon": "stadium_lon"})

    preferred = ["date", "phase", "stage_detail", "group", "match", "country", "city", "state_region",
                 "stadium", "stadium_lat", "stadium_lon"]
    merged = merged[[c for c in preferred if c in merged.columns] + [c for c in merged.columns if c not in preferred]]

    merged.to_csv(out_matches_csv, index=False)

    print("Wrote:")
    print(f" - {out_coords_csv}")
    print(f" - {out_matches_csv}")
    print(f"Matches parsed: {len(matches)}")
    print(f"Matches with coords: {merged['stadium_lat'].notna().sum()} / {len(merged)}")

    if also_write_databricks_filestore:
        merged.to_csv("/dbfs/FileStore/worldcup2026_matches_with_stadium_coords.csv", index=False)
        coords_out.to_csv("/dbfs/FileStore/worldcup2026_stadium_coords.csv", index=False)
        print("Also wrote to Databricks FileStore:")
        print(" - /dbfs/FileStore/worldcup2026_matches_with_stadium_coords.csv")
        print(" - /dbfs/FileStore/worldcup2026_stadium_coords.csv")


if __name__ == "__main__":
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop and loop.is_running():
        loop.create_task(main_async())
    else:
        asyncio.run(main_async())